In [1]:
# Understand that this entire program is based upon one primary idea - all data manipulation is done in a Pandas Dataframe
# A lot of people seem to have a problem with that - Why do you do it in a pandas dataframe.
# For starters, it's easy! The syntax is easy to use and remember.
# Also, the pandas dataframe was invented for the purpose of time series manipulation! So every other argument after this fails. 

In [2]:
### Main program - get all the data from kite connect and start manipualtion

In [2]:
from kiteconnect import KiteConnect
import pandas as pd
from selenium import webdriver
import time
import os
from datetime import date
pd.options.mode.chained_assignment = None

In [157]:
token_path = 'api_key.txt'
key_secret = open(token_path,'r').read().split()   #gives a list object
kite = KiteConnect(api_key = key_secret[0])
#virtually start the chromedriver -  don't make the window pop up
service = webdriver.chrome.service.Service('C:/Users/crypt/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe')
service.start()
#don't render the browser, save some RAM
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#just aggregating all the capabilities like headless, memoryless etc
options = options.to_capabilities()
#start the driver, pass the options 
driver = webdriver.Remote(service.service_url,options)
driver.get(kite.login_url())
driver.implicitly_wait(2)
#fint the element in the webpage by the HTML path
username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
username.send_keys(key_secret[2])
password.send_keys(key_secret[3])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
pin.send_keys(key_secret[4])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
time.sleep(2)
request_token = driver.current_url.split('=')[1].split('&action')[0]
with open('request_token.txt','w') as file:
    file.write(request_token)
driver.quit()

#Use request_token and get_access_token
request_token = open('request_token.txt','r').read().split()
key_secret = open('api_key.txt','r').read().split()
kite = KiteConnect(api_key = key_secret[0])
data = kite.generate_session(request_token[0],api_secret = key_secret[1])
with open('access_token.txt','w') as file:
    file.write(data['access_token'])
    
# Set access token and you're all set!!!!!!!!!!   
kite.set_access_token(open('access_token.txt','r').read().split()[0])

In [158]:
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,258313,1009,CPSE INDEX,CPSE INDEX,0.0,,0.0,0.00,0,EQ,INDICES,NSE
1,264713,1034,HANGSENG BEES-NAV,HANGSENG BEES-NAV,0.0,,0.0,0.00,0,EQ,INDICES,NSE
2,264969,1035,INDIA VIX,INDIA VIX,0.0,,0.0,0.00,0,EQ,INDICES,NSE
3,258057,1008,LIX 15,LIX 15,0.0,,0.0,0.00,0,EQ,INDICES,NSE
4,258569,1010,NI15,NI15,0.0,,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
2507,978945,3824,ZODJRDMKJ,ZODIAC JRD-MKJ,0.0,,0.0,0.05,1,EQ,NSE,NSE
2508,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.0,0.05,1,EQ,NSE,NSE
2509,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.0,0.05,1,EQ,NSE,NSE
2510,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,,0.0,0.05,1,EQ,NSE,NSE


In [231]:
companies_nse = ['SBIN']

In [233]:
def instrumentLookup(symbol):
    global instrument_df
    try: 
        return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]
    except:
        return -1

In [234]:
def fetchOHLC(ticker,interval,start_date,end_date):
    instrument  = instrumentLookup(instrument_df,ticker)
    #Both the date and the interval need to be passed to historical_data in string format
    # This is contradictory to NSEpy
    data = pd.DataFrame(kite.historical_data(instrument,start_date,end_date,interval))
    data.set_index('date',inplace = True)
    return data

In [235]:
tokens_nse = []
for i in range(len(companies_nse)):
    tokens_nse.append(instrumentLookup(companies_nse[i]))
    

In [236]:
tokens_nse

[779521]

In [224]:
import statistics as stats
class technicalIndicators:
    
    @classmethod
    def simpleMovingAverage(cls,data,timePeriod):
        history = []
        sma_values = []
        
        for close_price in data.close:
            history.append(close_price)
            if len(history) > timePeriod:
                del(history[0])
            sma_values.append(stats.mean(history))
        return sma_values
    
    @classmethod
    def adx(cls,data,n):
        
        df2 = data.copy()
        df2['H-L']=abs(df2['high']-df2['low'])
        df2['H-PC']=abs(df2['high']-df2['close'].shift(1))
        df2['L-PC']=abs(df2['low']-df2['close'].shift(1))
        df2['TR']=df2[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
        df2['DMplus']=np.where((df2['high']-df2['high'].shift(1))>(df2['low'].shift(1)-df2['low']),df2['high']-df2['high'].shift(1),0)
        df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
        df2['DMminus']=np.where((df2['low'].shift(1)-df2['low'])>(df2['high']-df2['high'].shift(1)),df2['low'].shift(1)-df2['low'],0)
        df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
        TRn = []
        DMplusN = []
        DMminusN = []
        TR = df2['TR'].tolist()
        DMplus = df2['DMplus'].tolist()
        DMminus = df2['DMminus'].tolist()
        for i in range(len(df2)):
            if i < n:
                TRn.append(np.NaN)
                DMplusN.append(np.NaN)
                DMminusN.append(np.NaN)
            elif i == n:
                TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
                DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
                DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
            elif i > n:
                TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
                DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
                DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
        df2['TRn'] = np.array(TRn)
        df2['DMplusN'] = np.array(DMplusN)
        df2['DMminusN'] = np.array(DMminusN)
        df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
        df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
        df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
        df2['DIsum']=df2['DIplusN']+df2['DIminusN']
        df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
        ADX = []
        DX = df2['DX'].tolist()
        for j in range(len(df2)):
            if j < 2*n-1:
                ADX.append(np.NaN)
            elif j == 2*n-1:
                ADX.append(df2['DX'][j-n+1:j+1].mean())
            elif j > 2*n-1:
                ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
        df2['ADX']=np.array(ADX)
        return df2['ADX']
   
    
        

In [225]:
#This will actually be a file of multiple strategies
class strategies:
    
    @classmethod
    def fiftyTwoHundredCrossoverSMA(cls,data):
        fifty_sma = technicalIndicators.simpleMovingAverage(data,50)
        two_hundred_sma = technicalIndicators.simpleMovingAverage(data,200)
        data = data.assign(fifty_sma = pd.Series(fifty_sma,index = data.index), 
                           twohundred_sma = pd.Series(two_hundred_sma,index = data.index))
        
        
        signals = []
        s1 = 0
        for i in range(len(data)):
            if (((data['fifty_sma'].iloc[i]>data['twohundred_sma'].iloc[i]) and (data['close'].iloc[i]>data['fifty_sma'].iloc[i])) and s1 == 0):
                signals.append(1)
                s1 = 1
            elif (((data['fifty_sma'].iloc[i]<data['twohundred_sma'].iloc[i]) or (data['close'].iloc[i]<data['fifty_sma'].iloc[i])) and s1 == 1):
                signals.append(-1)
                s1 = 0
                
            else:
                signals.append(0)
                
        data = data.assign(signals = pd.Series(signals,index = data.index))
            
        
        return data
    
    @classmethod
    def hitAndRun(cls,data): #All the strategies can be used individually but I am using only as an addition
        ##ADX > 30, stock price > 10 day sma and 50 day sma
        adx = technicalIndicators.adx(data,14)
        data['adx'] = adx
        data['adx'] = data['adx'].fillna(0)
        
        
        #Expansion Breakouts
        exapnsion_breakout = []
        s1 = 0
        for i in range(len(data)):
            if data['high'][i] > data['high'][-59:i].max():
                if (data["high"][i]-data["low"][i]>=(data["high"][-10:i]-data["low"][-10:i]).max()) and s1 == 0:
                    exapnsion_breakout.append(1)
                    s1 = 1

            elif data["low"][i] < data["low"][-59:i].min():
                if (data["high"][i] - data["low"][i] >= (data["high"][-10:i] - data["low"][-10:i]).max()) and s1 == 1:
                    exapnsion_breakout.append(-1)
                    s1 = 0

            else:
                exapnsion_breakout.append(0)
                
        
        #data = data.assign(expansion_breakout = pd.Series(expansion_breakout,index = data.index))   
            
                
        
        return exapnsion_breakout
        
    
    

In [226]:
class PnL:
    
    @classmethod
    def unrealizedProfitLoss(cls,data):
        data = data[(data['signals'] == 1 ) | (data['signals'] == -1)]
        pos = 0
        neg = 0
        if len(data)%2 != 0:
            data.drop(data.tail(1).index,inplace=True) # delete last row of a pandas dataframe
            
        for i in range(len(data)):
            if data['signals'].iloc[i] == 1:
                pos = pos + data['close'].iloc[i]
            elif data['signals'].iloc[i] == -1:
                neg = neg + data['close'].iloc[i] 
                
        return neg-pos

In [227]:
class instrument:
    def __init__(self,instrumentToken,start_date,end_date,interval):
        self.instrumentToken = instrumentToken 
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.data = historicalData.getData(self.instrumentToken,self.start_date,self.end_date,self.interval)
        # change strategy here to whatever strategy you want to apply
        self.strategy = strategies.hitAndRun(self.data)
        #self.pnl = PnL.unrealizedProfitLoss(self.strategy)
        

In [228]:
class historicalData:
    
    @classmethod
    def getData(cls,instrument_token,start_date,end_date,interval):
        data = pd.DataFrame(kite.historical_data(instrument_token,start_date,end_date,interval))
        data.set_index('date',inplace = True)
        return data
     

In [237]:
inst = instrument(779521,'2017-02-15','2020-02-15','day')

In [179]:
inst.pnl

AttributeError: 'instrument' object has no attribute 'pnl'

In [1]:
#inst.strategy

In [116]:
data = inst.strategy[(inst.strategy['signals'] == 1 ) | (inst.strategy['signals'] == -1)]

In [117]:
data

,open,high,low,close,volume,fifty_sma,twohundred_sma,signals
date,,,,,,,,
2016-12-15 00:00:00+05:30,259.90,267.50,259.00,265.40,12274497,259.725,259.638235,1
2016-12-20 00:00:00+05:30,261.85,262.90,253.25,254.55,12765309,259.790,259.670370,-1
2017-01-18 00:00:00+05:30,257.10,259.30,256.60,258.35,8048897,257.310,256.766000,1
2017-01-20 00:00:00+05:30,258.50,260.50,250.30,251.05,15341756,256.673,256.712987,-1
2017-02-16 00:00:00+05:30,277.00,277.00,269.65,270.50,21794688,259.124,259.085263,1
2017-05-24 00:00:00+05:30,290.05,290.45,281.65,283.15,16435958,288.479,269.256918,-1
2017-05-25 00:00:00+05:30,284.15,291.35,283.55,290.10,15018467,288.840,269.387187,1
2017-05-26 00:00:00+05:30,290.50,291.00,285.30,288.45,12304394,289.116,269.505590,-1
2017-06-21 00:00:00+05:30,288.25,292.45,288.25,290.90,10537576,290.807,271.291061,1


In [89]:
len(data)

13

In [91]:
data.drop(data.tail(1).index,inplace=True)

C:\Users\MAHE\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [92]:
data

,open,high,low,close,volume,fifty_sma,twohundred_sma,signals
date,,,,,,,,
2017-02-14 00:00:00+05:30,517.50,526.45,514.95,524.90,8340828,524.6774,521.053118,1
2017-05-18 00:00:00+05:30,670.13,675.00,662.05,663.68,4475832,669.8238,573.720000,-1
2017-05-29 00:00:00+05:30,668.50,682.73,667.80,678.45,10982340,671.6092,577.457222,1
2017-05-31 00:00:00+05:30,683.50,683.98,667.90,670.35,7508198,672.6262,578.646951,-1
2017-06-14 00:00:00+05:30,657.95,680.00,657.95,678.75,24202054,678.4608,583.589368,1
2017-09-27 00:00:00+05:30,825.60,825.70,795.50,799.15,8434196,804.6412,669.648150,-1
2017-10-04 00:00:00+05:30,799.20,824.95,795.60,818.85,8643924,806.6646,675.182250,1
2017-12-21 00:00:00+05:30,918.45,923.95,910.00,912.95,4147162,918.6250,777.607500,-1
2017-12-22 00:00:00+05:30,914.65,922.40,909.30,920.40,4872675,919.5780,778.946100,1


In [85]:

for i in range(len(inst.strategy)):
    if inst.strategy['signals'].iloc[i] == 1:
        total = total - inst.strategy['close'].iloc[i]
    elif inst.strategy['signals'].iloc[i] == -1:
        total = inst.strategy['close'].iloc[i] + total 
   
        

In [86]:
total

-589.02

In [94]:
pos = 0
neg = 0
for i in range(len(data)):
    if data['signals'].iloc[i] == 1:
        pos = pos + data['close'].iloc[i]
    elif data['signals'].iloc[i] == -1:
        neg = neg + data['close'].iloc[i] 

In [95]:
neg-pos

342.27999999999975

## Pattern scanner

In [ ]:
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(ChromeDriverManager().install())

In [12]:
token_path = 'api_key.txt'
key_secret = open(token_path,'r').read().split()   #gives a list object
kite = KiteConnect(api_key = key_secret[0])
#virtually start the chromedriver -  don't make the window pop up
service = webdriver.chrome.service.Service('C:/Users/crypt/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe')
service.start()
#don't render the browser, save some RAM
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#just aggregating all the capabilities like headless, memoryless etc
options = options.to_capabilities()
#start the driver, pass the options 
driver = webdriver.Remote(service.service_url,options)
driver.get(kite.login_url())
driver.implicitly_wait(2)
#fint the element in the webpage by the HTML path
username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
username.send_keys(key_secret[2])
password.send_keys(key_secret[3])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
pin.send_keys(key_secret[4])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
time.sleep(2)
request_token = driver.current_url.split('=')[1].split('&action')[0]
with open('request_token.txt','w') as file:
    file.write(request_token)
driver.quit()

#Use request_token and get_access_token
request_token = open('request_token.txt','r').read().split()
key_secret = open('api_key.txt','r').read().split()
kite = KiteConnect(api_key = key_secret[0])
data = kite.generate_session(request_token[0],api_secret = key_secret[1])
with open('access_token.txt','w') as file:
    file.write(data['access_token'])
    
# Set access token and you're all set!!!!!!!!!!   
kite.set_access_token(open('access_token.txt','r').read().split()[0])

In [ ]:
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)

def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

def doji(ohlc_df):    
    """returns dataframe with doji candle column"""
    df = ohlc_df.copy()
    avg_candle_size = abs(df["close"] - df["open"]).median()
    df["doji"] = abs(df["close"] - df["open"]) <=  (0.05 * avg_candle_size)
    return df

def maru_bozu(ohlc_df):    
    """returns dataframe with maru bozu candle column"""
    df = ohlc_df.copy()
    avg_candle_size = abs(df["close"] - df["open"]).median()
    df["h-c"] = df["high"]-df["close"]
    df["l-o"] = df["low"]-df["open"]
    df["h-o"] = df["high"]-df["open"]
    df["l-c"] = df["low"]-df["close"]
    df["maru_bozu"] = np.where((df["close"] - df["open"] > 2*avg_candle_size) & \
                               (df[["h-c","l-o"]].max(axis=1) < 0.005*avg_candle_size),"maru_bozu_green",
                               np.where((df["open"] - df["close"] > 2*avg_candle_size) & \
                               (abs(df[["h-o","l-c"]]).max(axis=1) < 0.005*avg_candle_size),"maru_bozu_red",False))
    df.drop(["h-c","l-o","h-o","l-c"],axis=1,inplace=True)
    return df

def hammer(ohlc_df):    
    """returns dataframe with hammer candle column"""
    df = ohlc_df.copy()
    df["hammer"] = (((df["high"] - df["low"])>3*(df["open"] - df["close"])) & \
                   ((df["close"] - df["low"])/(.001 + df["high"] - df["low"]) > 0.6) & \
                   ((df["open"] - df["low"])/(.001 + df["high"] - df["low"]) > 0.6)) & \
                   (abs(df["close"] - df["open"]) > 0.1* (df["high"] - df["low"]))
    return df


def shooting_star(ohlc_df):    
    """returns dataframe with shooting star candle column"""
    df = ohlc_df.copy()
    df["sstar"] = (((df["high"] - df["low"])>3*(df["open"] - df["close"])) & \
                   ((df["high"] - df["close"])/(.001 + df["high"] - df["low"]) > 0.6) & \
                   ((df["high"] - df["open"])/(.001 + df["high"] - df["low"]) > 0.6)) & \
                   (abs(df["close"] - df["open"]) > 0.1* (df["high"] - df["low"]))
    return df

def levels(ohlc_day):    
    """returns pivot point and support/resistance levels"""
    high = round(ohlc_day["high"][-1],2)
    low = round(ohlc_day["low"][-1],2)
    close = round(ohlc_day["close"][-1],2)
    pivot = round((high + low + close)/3,2)
    r1 = round((2*pivot - low),2)
    r2 = round((pivot + (high - low)),2)
    r3 = round((high + 2*(pivot - low)),2)
    s1 = round((2*pivot - high),2)
    s2 = round((pivot - (high - low)),2)
    s3 = round((low - 2*(high - pivot)),2)
    return (pivot,r1,r2,r3,s1,s2,s3)

def trend(ohlc_df,n):
    "function to assess the trend by analyzing each candle"
    df = ohlc_df.copy()
    df["up"] = np.where(df["low"]>=df["low"].shift(1),1,0)
    df["dn"] = np.where(df["high"]<=df["high"].shift(1),1,0)
    if df["close"][-1] > df["open"][-1]:
        if df["up"][-1*n:].sum() >= 0.7*n:
            return "uptrend"
    elif df["open"][-1] > df["close"][-1]:
        if df["dn"][-1*n:].sum() >= 0.7*n:
            return "downtrend"
    else:
        return None
   
def res_sup(ohlc_df,ohlc_day):
    """calculates closest resistance and support levels for a given candle"""
    level = ((ohlc_df["close"][-1] + ohlc_df["open"][-1])/2 + (ohlc_df["high"][-1] + ohlc_df["low"][-1])/2)/2
    p,r1,r2,r3,s1,s2,s3 = levels(ohlc_day)
    l_r1=level-r1
    l_r2=level-r2
    l_r3=level-r3
    l_p=level-p
    l_s1=level-s1
    l_s2=level-s2
    l_s3=level-s3
    lev_ser = pd.Series([l_p,l_r1,l_r2,l_r3,l_s1,l_s2,l_s3],index=["p","r1","r2","r3","s1","s2","s3"])
    sup = lev_ser[lev_ser>0].idxmin()
    res = lev_ser[lev_ser<0].idxmax()
    return (eval('{}'.format(res)), eval('{}'.format(sup)))

def candle_type(ohlc_df):    
    """returns the candle type of the last candle of an OHLC DF"""
    candle = None
    if doji(ohlc_df)["doji"][-1] == True:
        candle = "doji"    
    if maru_bozu(ohlc_df)["maru_bozu"][-1] == "maru_bozu_green":
        candle = "maru_bozu_green"       
    if maru_bozu(ohlc_df)["maru_bozu"][-1] == "maru_bozu_red":
        candle = "maru_bozu_red"        
    if shooting_star(ohlc_df)["sstar"][-1] == True:
        candle = "shooting_star"        
    if hammer(ohlc_df)["hammer"][-1] == True:
        candle = "hammer"       
    return candle

def candle_pattern(ohlc_df,ohlc_day):    
    """returns the candle pattern identified"""
    pattern = None
    signi = "low"
    avg_candle_size = abs(ohlc_df["close"] - ohlc_df["open"]).median()
    sup, res = res_sup(ohlc_df,ohlc_day)
    
    if (sup - 1.5*avg_candle_size) < ohlc_df["close"][-1] < (sup + 1.5*avg_candle_size):
        signi = "HIGH"
        
    if (res - 1.5*avg_candle_size) < ohlc_df["close"][-1] < (res + 1.5*avg_candle_size):
        signi = "HIGH"
    
    if candle_type(ohlc_df) == 'doji' \
        and ohlc_df["close"][-1] > ohlc_df["close"][-2] \
        and ohlc_df["close"][-1] > ohlc_df["open"][-1]:
            pattern = "doji_bullish"
    
    if candle_type(ohlc_df) == 'doji' \
        and ohlc_df["close"][-1] < ohlc_df["close"][-2] \
        and ohlc_df["close"][-1] < ohlc_df["open"][-1]:
            pattern = "doji_bearish" 
            
    if candle_type(ohlc_df) == "maru_bozu_green":
        pattern = "maru_bozu_bullish"
    
    if candle_type(ohlc_df) == "maru_bozu_red":
        pattern = "maru_bozu_bearish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "uptrend" and candle_type(ohlc_df) == "hammer":
        pattern = "hanging_man_bearish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "downtrend" and candle_type(ohlc_df) == "hammer":
        pattern = "hammer_bullish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "uptrend" and candle_type(ohlc_df) == "shooting_star":
        pattern = "shooting_star_bearish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "uptrend" \
        and candle_type(ohlc_df) == "doji" \
        and ohlc_df["high"][-1] < ohlc_df["close"][-2] \
        and ohlc_df["low"][-1] > ohlc_df["open"][-2]:
        pattern = "harami_cross_bearish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "downtrend" \
        and candle_type(ohlc_df) == "doji" \
        and ohlc_df["high"][-1] < ohlc_df["open"][-2] \
        and ohlc_df["low"][-1] > ohlc_df["close"][-2]:
        pattern = "harami_cross_bullish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "uptrend" \
        and candle_type(ohlc_df) != "doji" \
        and ohlc_df["open"][-1] > ohlc_df["high"][-2] \
        and ohlc_df["close"][-1] < ohlc_df["low"][-2]:
        pattern = "engulfing_bearish"
        
    if trend(ohlc_df.iloc[:-1,:],7) == "downtrend" \
        and candle_type(ohlc_df) != "doji" \
        and ohlc_df["close"][-1] > ohlc_df["high"][-2] \
        and ohlc_df["open"][-1] < ohlc_df["low"][-2]:
        pattern = "engulfing_bullish"
       
    return "Significance - {}, Pattern - {}".format(signi,pattern)

##############################################################################################
tickers = ["ZEEL","WIPRO","VEDL","ULTRACEMCO","UPL","TITAN","TECHM","TATASTEEL",
           "TATAMOTORS","TCS","SUNPHARMA","SBIN","SHREECEM","RELIANCE","POWERGRID",
           "ONGC","NESTLEIND","NTPC","MARUTI","M&M","LT","KOTAKBANK","JSWSTEEL","INFY",
           "INDUSINDBK","IOC","ITC","ICICIBANK","HDFC","HINDUNILVR","HINDALCO",
           "HEROMOTOCO","HDFCBANK","HCLTECH","GRASIM","GAIL","EICHERMOT","DRREDDY",
           "COALINDIA","CIPLA","BRITANNIA","INFRATEL","BHARTIARTL","BPCL","BAJAJFINSV",
           "BAJFINANCE","BAJAJ-AUTO","AXISBANK","ASIANPAINT","ADANIPORTS","IDEA",
           "MCDOWELL-N","UBL","NIACL","SIEMENS","SRTRANSFIN","SBILIFE","PNB",
           "PGHH","PFC","PEL","PIDILITIND","PETRONET","PAGEIND","OFSS","NMDC","NHPC",
           "MOTHERSUMI","MARICO","LUPIN","L&TFH","INDIGO","IBULHSGFIN","ICICIPRULI",
           "ICICIGI","HINDZINC","HINDPETRO","HAVELLS","HDFCLIFE","HDFCAMC","GODREJCP",
           "GICRE","DIVISLAB","DABUR","DLF","CONCOR","COLPAL","CADILAHC","BOSCHLTD",
           "BIOCON","BERGEPAINT","BANKBARODA","BANDHANBNK","BAJAJHLDNG","DMART",
           "AUROPHARMA","ASHOKLEY","AMBUJACEM","ADANITRANS","ACC",
           "WHIRLPOOL","WABCOINDIA","VOLTAS","VINATIORGA","VBL","VARROC","VGUARD",
           "UNIONBANK","UCOBANK","TRENT","TORNTPOWER","TORNTPHARM","THERMAX","RAMCOCEM",
           "TATAPOWER","TATACONSUM","TVSMOTOR","TTKPRESTIG","SYNGENE","SYMPHONY",
           "SUPREMEIND","SUNDRMFAST","SUNDARMFIN","SUNTV","STRTECH","SAIL","SOLARINDS",
           "SHRIRAMCIT","SCHAEFFLER","SANOFI","SRF","SKFINDIA","SJVN","RELAXO",
           "RAJESHEXPO","RECLTD","RBLBANK","QUESS","PRESTIGE","POLYCAB","PHOENIXLTD",
           "PFIZER","PNBHOUSING","PIIND","OIL","OBEROIRLTY","NAM-INDIA","NATIONALUM",
           "NLCINDIA","NBCC","NATCOPHARM","MUTHOOTFIN","MPHASIS","MOTILALOFS","MINDTREE",
           "MFSL","MRPL","MANAPPURAM","MAHINDCIE","M&MFIN","MGL","MRF","LTI","LICHSGFIN",
           "LTTS","KANSAINER","KRBL","JUBILANT","JUBLFOOD","JINDALSTEL","JSWENERGY",
           "IPCALAB","NAUKRI","IGL","IOB","INDHOTEL","INDIANB","IBVENTURES","IDFCFIRSTB",
           "IDBI","ISEC","HUDCO","HONAUT","HAL","HEXAWARE","HATSUN","HEG","GSPL",
           "GUJGASLTD","GRAPHITE","GODREJPROP","GODREJIND","GODREJAGRO","GLENMARK",
           "GLAXO","GILLETTE","GMRINFRA","FRETAIL","FCONSUMER","FORTIS","FEDERALBNK",
           "EXIDEIND","ESCORTS","ERIS","ENGINERSIN","ENDURANCE","EMAMILTD","EDELWEISS",
           "EIHOTEL","LALPATHLAB","DALBHARAT","CUMMINSIND","CROMPTON","COROMANDEL","CUB",
           "CHOLAFIN","CHOLAHLDNG","CENTRALBK","CASTROLIND","CANBK","CRISIL","CESC",
           "BBTC","BLUEDART","BHEL","BHARATFORG","BEL","BAYERCROP","BATAINDIA",
           "BANKINDIA","BALKRISIND","ATUL","ASTRAL","APOLLOTYRE","APOLLOHOSP",
           "AMARAJABAT","ALKEM","APLLTD","AJANTPHARM","ABFRL","ABCAPITAL","ADANIPOWER",
           "ADANIGREEN","ADANIGAS","ABBOTINDIA","AAVAS","AARTIIND","AUBANK","AIAENG","3MINDIA"]


def main():
    for ticker in tickers:
        try:
            ohlc = fetchOHLC(ticker, '5minute',5)
            ohlc_day = fetchOHLC(ticker, 'day',30) 
            ohlc_day = ohlc_day.iloc[:-1,:]       
            cp = candle_pattern(ohlc,ohlc_day) 
            print(ticker, ": ",cp)   
        except:
            print("skipping for ",ticker)
        
# Continuous execution        
starttime=time.time()
timeout = time.time() + 60*60*1  # 60 seconds times 60 meaning the script will run for 1 hr
while time.time() <= timeout:
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main()
        time.sleep(300 - ((time.time() - starttime) % 300.0)) # 300 second interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()

passthrough at  2020-06-25 13:43:54
skipping for  ZEEL
skipping for  WIPRO
skipping for  VEDL
skipping for  ULTRACEMCO
skipping for  UPL
skipping for  TITAN
skipping for  TECHM
skipping for  TATASTEEL
skipping for  TATAMOTORS
skipping for  TCS
skipping for  SUNPHARMA
skipping for  SBIN
skipping for  SHREECEM
skipping for  RELIANCE
skipping for  POWERGRID
skipping for  ONGC
skipping for  NESTLEIND
skipping for  NTPC
skipping for  MARUTI
skipping for  M&M
skipping for  LT
skipping for  KOTAKBANK
skipping for  JSWSTEEL
skipping for  INFY
skipping for  INDUSINDBK
skipping for  IOC
skipping for  ITC
skipping for  ICICIBANK
skipping for  HDFC
skipping for  HINDUNILVR
skipping for  HINDALCO
skipping for  HEROMOTOCO
skipping for  HDFCBANK
skipping for  HCLTECH
skipping for  GRASIM
skipping for  GAIL
skipping for  EICHERMOT
skipping for  DRREDDY
skipping for  COALINDIA
skipping for  CIPLA
skipping for  BRITANNIA
skipping for  INFRATEL
skipping for  BHARTIARTL
skipping for  BPCL
skipping for  BA

## Backtesting with zipline